In [ ]:
import requests

from bs4 import BeautifulSoup

import pandas as pd

base_url = "https://www.amazon.in/gp/bestsellers/books/4149461031/ref=zg_bs_pg_{}?ie=UTF8&pg={}"

headers = {

    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"

}



book_list = []

for page in range(1,6):

    # construct the URL for the current page

    url=base_url.format(page, page)



    # send a GET request to the URL

    response = requests.get(url, headers=headers)



    # parse the HTML content of the page

    soup = BeautifulSoup(response.content, "html.parser")



    # find all book entries on the page

    books = soup.find_all("div", class_="zg-grid-general-faceout")



    # iterate through each book entry and extract details

    for book in books:

        if len(book_list) < 50:  # stop once we've collected 50 books

            author = book.find("a", class_="a-size-small a-link-child").get_text(strip=True) if book.find("a", class_="a-size-small a-link-child") else "N/A"

            rating = book.find("span", class_="a-icon-alt").get_text(strip=True) if book.find("span", class_="a-icon-alt") else "N/A"



            # append the extracted data

            book_list.append({"Author": author,

                            "Rating": rating})

        else:

            break

df=pd.DataFrame(book_list)

print(df)

df.to_csv("amazon_top_50_books_authors_ratings.csv", index=False)

print(f"Saved {len(df)} reviews to amazon_reviews_sample.csv")

                                         Author              Rating
0                                      PR Yadav  4.4 out of 5 stars
1                                           N/A  4.5 out of 5 stars
2                                   James Allen  4.4 out of 5 stars
3                                     DC Pandey  4.2 out of 5 stars
4                                           N/A  4.4 out of 5 stars
5                                           N/A  4.6 out of 5 stars
6                              Shaurrya Singhvi                 N/A
7   Sachin Kumarr Laxman Prasad , Anjali Sharma  4.5 out of 5 stars
8                        ALLEN Expert Faculties  4.3 out of 5 stars
9                                           N/A  4.5 out of 5 stars
10                                 Preeti Gupta  4.4 out of 5 stars
11                                          N/A  4.2 out of 5 stars
12                       ALLEN Expert Faculties  4.1 out of 5 stars
13                           Wonder House Books 

In [4]:
import requests

from bs4 import BeautifulSoup

import pandas as pd

import time

import random


PRODUCT_SLUG = "itmfy2jxm4h87ptd"  # Slug from the Flipkart review URL

PRODUCT_PID = "SUGFY2JXQQKRYFBP"  # PID from the Flipkart review URL

HEADERS = {

    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",

    "Accept-Language": "en-US,en;q=0.9",

}


def get_reviews(page_no):

    # Flipkart review URL structure

    url = f"https://www.flipkart.com/flipkart/product-reviews/{PRODUCT_SLUG}?pid={PRODUCT_PID}&page={page_no}"



    response = requests.get(url, headers=HEADERS)

    if response.status_code != 200:

        print(f"Failed to fetch page {page_no}: {response.status_code}")

        return []



    soup = BeautifulSoup(response.text, "lxml")

    reviews = []



    for review_block in soup.find_all("div", class_="_2MIQdG"):

        review = {}

        # Review Title

        review["review_title"] = review_block.find("p", class_="_2-N8zT").text.strip() if review_block.find("p", class_="_2-N8zT") else "N/A"



        rating_element = review_block.find("div", class_="_3LWZlK")

        review["rating"] = rating_element.text.strip() if rating_element else "N/A"

        # Review Body

        review["review_body"] = review_block.find("div", class_="t-ZTKy").text.strip() if review_block.find("div", class_="t-ZTKy") else "N/A"

        # Author and Date are often in the same parent element

        author_date_element = review_block.find("p", class_="_2sc7ZR")

        if author_date_element:

            spans = author_date_element.find_all("span")

            if len(spans) >= 1:

                review["author"] = spans[0].text.replace("Verified Buyer", "").strip() if "Verified Buyer" in spans[0].text else spans[0].text.strip()

            else:

                review["author"] = "N/A"


            date_text = "N/A"

            for span in spans:

                if "Reviewed on" in span.text:

                    date_text = span.text.replace("Reviewed on", "").strip()

                    break

                elif len(spans) > 1 and spans.index(span) == 1: # If there's a second span, assume it's date

                    date_text = span.text.strip()

                    break

            review["date"] = date_text

        else:

            review["author"] = "N/A"

            review["date"] = "N/A"



        reviews.append(review)

    return reviews


def scrape_flipkart_reviews(pages=3):

    all_reviews = []

    for p in range(1, pages+1):

        print(f"Fetching page {p}...")

        reviews = get_reviews(p)

        all_reviews.extend(reviews)

        time.sleep(random.uniform(1, 3))
    return all_reviews



if __name__ == "__main__":

    reviews_data = scrape_flipkart_reviews(pages=3)

    df = pd.DataFrame(reviews_data)

    df.to_csv("flipkart_reviews_sample.csv", index=False, encoding="utf-8-sig")

    print(f"Saved {len(df)} reviews to flipkart_reviews_sample.csv")

Fetching page 1...
Failed to fetch page 1: 529
Fetching page 2...
Failed to fetch page 2: 529
Fetching page 3...
Failed to fetch page 3: 529
Saved 0 reviews to flipkart_reviews_sample.csv
